In [1]:
import numpy as np

In [2]:
import tools

In [6]:
# Replace with a loop over subjects:
subject = '856766'

In [7]:
foo = tools.compare_models(subject)

Files don't exist - going ahead


KeyboardInterrupt: 

In [5]:
foo

('992774', True)

In [ ]:
#tools.save_wm_mask

In [ ]:
# t1 = time.time()
# compare_models(subject)
# t2 = time.time()
# delta_t = t2 - t1

In [ ]:
delta_t / 60